# Tutorial about regions

Regions define a support for localization data or specify a hull that captures a set of localizations. Locan provides various region classes with a standard set of attributes and methods.

In [ ]:
%matplotlib inline

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import locan as lc

In [ ]:
lc.show_versions(system=False, dependencies=False, verbose=False)

## Region definitions

The standard set of attributes and methods is defined by the abstract base class `Region` that all region classes inherit.

In [ ]:
lc.Region?

In [ ]:
print("Methods:")
[method for method in dir(lc.Region) if not method.startswith('_')]

Further definitions can be found in the abstract classes `Region1D`, `Region2D` and `Region3D` and all specific region classes.

In [ ]:
import inspect
inspect.getmembers(lc.data.region, inspect.isabstract)

## Use Region classes

Use one of the following classes to define a region in 1, 2 or 3 dimensions:

In [ ]:
print("Empty Region:\n", [lc.EmptyRegion.__name__], "\n")
print("Regions in 1D:\n", [cls.__name__ for cls in lc.Region1D.__subclasses__()], "\n")
print("Regions in 2D:\n", [cls.__name__ for cls in lc.Region2D.__subclasses__()], "\n")
print("Regions in 3D:\n", [cls.__name__ for cls in lc.Region3D.__subclasses__()], "\n")
print("Regions in nD:\n", [cls.__name__ for cls in lc.RegionND.__subclasses__()], "\n")

The region constructors take different parameters. 

REMEMBER: Angles are taken in degrees.

In [ ]:
region = lc.Rectangle(corner=(0, 0), width=1, height=2, angle=45)
region

In [ ]:
points = ((0, 0), (0, 1), (1, 1), (1, 0.5), (0, 0))
holes = [((0.2, 0.2), (0.2, 0.4), (0.4, 0.4), (0.3, 0.25)), ((0.5, 0.5), (0.5, 0.8), (0.8, 0.8), (0.7, 0.45))]
region = lc.Polygon(points, holes)
print(region)
region

Several attributes are available, e.g. about the area or circumference.

In [ ]:
dict(dimension=region.dimension, bounds=region.bounds, extent=region.extent, bounding_box=region.bounding_box, centroid=region.centroid, max_distance=region.max_distance, 
     region_measure= region.region_measure, subregion_measure=region.subregion_measure)  

A list of points defining a polygon that resembles the region is available.

In [ ]:
print("Points:\n", region.points, "\n")
print("Holes:\n", region.holes)

Region can be constructed from interval tuples indicating feature ranges.

In [ ]:
region_1d = lc.Region.from_intervals((0, 1))
region_2d = lc.Region.from_intervals(((0, 1), (0, 1)))
region_3d = lc.Region.from_intervals([(0, 1)] * 3)
region_4d = lc.Region.from_intervals([(0, 1)] * 4)

In [ ]:
for region in (region_1d, region_2d, region_3d, region_4d):
    print(region)

## Plot regions

Regions can be plotted as patch in mathplotlib figures.

In [ ]:
points = ((0, 0), (0, 1), (1, 1), (1, 0.5), (0, 0))
holes = [((0.2, 0.2), (0.2, 0.4), (0.4, 0.4), (0.3, 0.25)), ((0.5, 0.5), (0.5, 0.8), (0.8, 0.8), (0.7, 0.45))]
region = lc.Polygon(points, holes)

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1)
ax.add_patch(region.as_artist(fill=False, color='Blue'))
ax.add_patch(region.bounding_box.as_artist(fill=False, color='Grey'))
ax.plot(*region.centroid, '*', color='Red')
ax.axis('equal')
plt.show()

## Intersection, union, difference of regions

Methods are provided to check for intersection, difference, union and membership.

In [ ]:
other_region = lc.Rectangle(corner=(0.5, 0.2), width=1.5, height=1.5, angle=45)
other_region.shapely_object

In [ ]:
result = region.intersection(other_region)
print(result)

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1)
ax.add_patch(result.as_artist(fill=True, color='Blue'))
ax.add_patch(region.as_artist(fill=False, color='Red'))
ax.add_patch(other_region.as_artist(fill=False, color='Green'))
ax.axis('equal')
plt.show()

In [ ]:
result = region.symmetric_difference(other_region)
print(result)

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1)
ax.add_patch(result.as_artist(fill=True, color='Blue'))
ax.add_patch(region.as_artist(fill=False, color='Red'))
ax.add_patch(other_region.as_artist(fill=False, color='Green'))
ax.axis('equal')
plt.show()

In [ ]:
result = region.union(other_region)
print(result)

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1)
ax.add_patch(result.as_artist(fill=True, color='Blue'))
ax.add_patch(region.as_artist(fill=False, color='Red'))
ax.add_patch(other_region.as_artist(fill=False, color='Green'))
ax.axis('equal')
plt.show()

## Check if point is in region

`Region` has a `contains` method to select points that are within the region.

In [ ]:
inside_indices = other_region.contains(region.points)
contained_points = region.points[inside_indices]

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1)
ax.scatter(*region.points.T, color='Grey')
ax.scatter(*contained_points.T, color='Black')
ax.add_patch(other_region.as_artist(fill=False, color='Blue'))
ax.axis('equal')
plt.show()

## LocData and regions

LocData bring various hulls that define regions. Also LocData typically has a unique region defined as support. This can e.g. result from the definition of a region of interest using the ROI function or as specified in a corresponding yaml file.

### Create data in region:

A random dataset is created within a specified region (for other methods see simulation tutorial).

In [ ]:
region = lc.Rectangle(corner=(0, 0), width=1, height=1, angle=45)
locdata = lc.simulate_uniform(n_samples=1000, region=region, seed=1)
locdata.print_summary()

In [ ]:
region

### Show scatter plots together with regions

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1)
locdata.data.plot.scatter(x='position_x', y='position_y', ax=ax, color='Blue', label='locdata')
ax.add_patch(locdata.region.as_artist(fill=False, color='Red'))
ax.add_patch(locdata.region.bounding_box.as_artist(fill=False, color='Blue'))
ax.axis('equal')
plt.show()

## Select localizations within regions

LocData can be selected for localizations being inside the region.

In [ ]:
region = lc.Ellipse(center=(0, 0.5), width=1, height=0.5, angle=45)
locdata_in_region = lc.select_by_region(locdata, region)
locdata_in_region.region

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1)
locdata.data.plot.scatter(x='position_x', y='position_y', ax=ax, color='Blue', label='locdata', alpha=0.1)
ax.add_patch(region.as_artist(fill=False, color='Red'))
locdata_in_region.data.plot.scatter(x='position_x', y='position_y', ax=ax, color='Red', label='locdata_in_region')
ax.plot(*region.centroid, '*', color='Green')
ax.axis('equal')
plt.show()

## Regions of interest

The Roi class is an object that defines a region of interest for a specific localization dataset. It is mostly used to save and reload regions of interest after having selected them interactively, e.g. in napari. It is therefore related to region specifications and a unique LocData object. 

Define a region of interest (roi):

In [ ]:
roi = lc.Roi(reference=locdata, region=lc.Ellipse(center=(0, 0.5), width=1, height=0.5, angle=80))
roi

Create new LocData instance by selecting localizations within a roi.

In [ ]:
locdata_roi = roi.locdata()

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1)
locdata.data.plot.scatter(x='position_x', y='position_y', ax=ax, color='Blue', label='locdata', alpha=0.1)
ax.add_patch(roi.region.as_artist(fill=False))
locdata_roi.data.plot.scatter(x='position_x', y='position_y', ax=ax, color='Red', label='locdata_roi')
ax.plot(*roi.region.centroid, '*', color='Green')
ax.axis('equal')
plt.show()

### ROI input/output

If you have prepared rois and saved them as roi.yaml file you can read that data back in:

In [ ]:
import tempfile
from pathlib import Path

with tempfile.TemporaryDirectory() as tmp_directory:
    file_path = Path(tmp_directory) / 'roi.yaml'

    roi.to_yaml(path=file_path)

    roi_new = lc.Roi.from_yaml(path = file_path)
    roi_new.reference = roi.reference
    
new_locdata = roi_new.locdata()
new_locdata.meta

In [ ]:
roi_new